## Step 1: Preprocess Data

In [46]:
import pandas as pd

# Load your dataset (replace with your file path)
# File path
file_path = r"C:\Users\sagni\Documents\Personal Files\Research\doi_10_5061_dryad_k0p2ngfhn__v20250410\Data_Clustered_Cleaned.csv"
# Read CSV into dataframe
df = pd.read_csv(file_path)
df['Detected'] = 1

# Create site × year detection matrix for a specific species
species = "brown bear"  # change as needed
df_species = df[df['Common_Name'] == species]

# Pivot to detection history (Cluster_Agglo as site, Year as survey)
detection_matrix = df_species.pivot_table(index='Cluster_Agglo',
                                          columns='Year',
                                          values='Detected',
                                          aggfunc='max').fillna(0)

C:\Users\sagni\AppData\Local\Temp\ipykernel_8844\851336757.py:7: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


## Step 2: Build Occupancy Model (using PyMC)

In [47]:
import pymc as pm
import numpy as np

# Detection matrix as numpy array
y = detection_matrix.values.astype(int)
n_sites, n_surveys = y.shape

with pm.Model() as model:
    # Priors
    psi = pm.Beta("psi", 1, 1)  # occupancy probability
    p = pm.Beta("p", 1, 1)      # detection probability

    # Latent occupancy state (z): 1 if occupied, else 0
    z = pm.Bernoulli("z", p=psi, shape=n_sites)

    # Observation model: detect if occupied and detection works
    y_obs = pm.Bernoulli("y_obs", p=z[:, None] * p, observed=y)

    trace = pm.sample(2000, tune=1000, return_inferencedata=True)


Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>NUTS: [psi, p]
>BinaryGibbsMetropolis: [z]


Output()

Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 50 seconds.
C:\Users\sagni\anaconda3\envs\AgenticAI\Lib\site-packages\arviz\stats\diagnostics.py:596: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


## Step 3: Add Environmental Covariates

In [48]:
# Site-level covariates (e.g., most common habitat per site)
habitat_mode = df.groupby('Cluster_Agglo')['Habitat'].agg(lambda x: x.mode()[0])
group_size_mean = df.groupby('Cluster_Agglo')['Group_Size'].mean()
year_median = df.groupby('Cluster_Agglo')['Year'].median()

# Merge into design matrix
X = pd.get_dummies(habitat_mode, drop_first=True)
X['Group_Size'] = group_size_mean
X['Year'] = year_median
X = X.loc[detection_matrix.index]  # align rows

# Standardize
X_std = (X - X.mean()) / X.std()


In [53]:
import pandas as pd
import numpy as np
import pymc as pm
import aesara.tensor as at

# Step 1: Preprocess Detection Matrix
# Ensure detection matrix (y) is int and no missing values
y = detection_matrix.values.astype(int)
n_sites, n_surveys = y.shape

# Step 2: Create Covariate Matrix
# Group site-level environmental covariates
habitat_mode = df.groupby('Cluster_Agglo')['Habitat'].agg(lambda x: x.mode().iloc[0])
group_size_mean = df.groupby('Cluster_Agglo')['Group_Size'].mean()
year_median = df.groupby('Cluster_Agglo')['Year'].median()

# Encode habitat as dummy variables
X = pd.get_dummies(habitat_mode, drop_first=True)
X['Group_Size'] = group_size_mean
X['Year'] = year_median
X = X.loc[detection_matrix.index]  # Ensure alignment

# Step 3: Clean and Standardize
X_std = X.fillna(0).replace([np.inf, -np.inf], 0)
X_std = (X_std - X_std.mean()) / X_std.std()
X_matrix = X_std.to_numpy()

# Print for verification
print("X shape:", X_matrix.shape)
print("First few rows:\n", X_std.head())

# Step 4: Build Model
with pm.Model() as cov_model:
    # Priors for occupancy covariates
    beta = pm.Normal("beta", mu=0, sigma=1, shape=X_matrix.shape[1])
    alpha = pm.Normal("alpha", mu=0, sigma=1)

    # Linear predictor for occupancy (psi)
    psi_logit = alpha + at.dot(X_matrix, beta)
    psi = pm.Deterministic("psi", pm.math.sigmoid(psi_logit))

    # Detection probability
    p = pm.Beta("p", alpha=1, beta=1)

    # Latent occupancy state
    z = pm.Bernoulli("z", p=psi, shape=n_sites)

    # Observation model (conditional on presence and detection)
    y_obs = pm.Bernoulli("y_obs", p=z[:, None] * p, observed=y)

    # Sampling: Use 1 core for clearer errors
    cov_trace = pm.sample(
        draws=2000,
        tune=1000,
        cores=1,
        return_inferencedata=True,
        target_accept=0.95,
    )


WARNING (aesara.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (aesara.configdefaults): g++ not detected!  Aesara will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set Aesara flags cxx to an empty string.


ModuleNotFoundError: No module named 'distutils.msvccompiler'